## OSU DS4S Data Science Club
### RNASeq Workflow

1. Setup Environment
1. Download SRA Files
1. Convert SRA to fastq
1. Index Genome
1. Run fastQC
1. Run trim_galore
1. Align Reads
1. Run FeatureCounts on each file
1. Combine counts
1. Run multiqc
1. Display html links for QC

In [37]:
import subprocess
import os
import glob
from multiprocessing import Process, current_process, Pool


# reusable function to run cmd, pipe stdout to console
# and return std out back for error checking

def run_cmd(cmd):
    p = subprocess.Popen(cmd,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT,shell=True)
    output = ""
    while True:
        line = p.stdout.readline().decode(encoding='UTF-8')
        if not line:
            break
        print (line.strip())
        output = output + line
    return output

In [64]:
MAXCORES = '24'
ENVCMD = "source activate rnaseq"
FASTQS = 'fastqs'
TFASTQS = 'trimmed_fastqs'
PFASTQS = 'paired_fastqs'
TPFASTQS = 'trimmed_paired_fastqs'
FASTQCOUT = 'fastQC_output'
TFASTQCOUT = 'trimmed_fastQC_output'
STARDIR = './star_output'
FCDIR = './featureCount_output'

# MAX CPUS For Owens should be 24 (28 possible but reserve a few for interactivity)
os.environ['MAXCORES'] = '24'

# Set Working Directory
os.environ['WORKDIR'] = os.getcwd()

# Set sra_toolkit cfg location.  Needed to overide default download directory
# Will default to data directory in the current working directory.
os.environ['VDB_CONFIG'] = os.path.expandvars("$WORKDIR/sra_toolkit.kfg")

# The data download directories
os.environ['DATA'] =  os.path.expandvars("$WORKDIR/data/sra")

# The results directories
os.environ['FASTQS'] = 'fastqs'
os.environ['TFASTQS'] = 'trimmed_fastqs'
os.environ['PFASTQS'] = 'paired_fastqs'
os.environ['TPFASTQS'] = 'trimmed_paired_fastqs'
os.environ['FASTQCOUT'] = 'fastQC_output'
os.environ['TFASTQCOUT'] = 'trimmed_fastQC_output'
os.environ['STARDIR'] = './star_output'
os.environ['FCDIR'] = './featureCount_output'

# The binary directories
os.environ['BASEDIR'] = '/fs/project/PAS1307'
os.environ['BINDIR'] = os.path.expandvars("$BASEDIR/bin")
os.environ['SRABINDIR'] = os.path.expandvars("$BASEDIR/bin/sratoolkit.2.8.2-1-ubuntu64/bin")
os.environ['TGBINDIR'] = os.path.expandvars("$BASEDIR/bin/trim_galore")

# The Genome directories
os.environ['GENOMEDIR'] = os.path.expandvars("$BASEDIR/genomes")
#os.environ['GENOME'] = 'Drosophila_melanogaster/Ensembl/BDGP5.25/'         
os.environ['GENOME'] = 'Mus_musculus/Ensembl/GRCm38' 
os.environ['WHOLEGENOMEDIR'] = os.path.expandvars("$GENOMEDIR/$GENOME/Sequence/WholeGenomeFasta")
os.environ['GENEANNOTATIONDIR'] = os.path.expandvars("$GENOMEDIR/$GENOME/Annotation/Genes")
os.environ['STARGENOMEDIR'] = os.path.expandvars("$GENOMEDIR/$GENOME/Sequence/StarIndex")

In [10]:
cmd = '''
mkdir -p $FASTQS
mkdir -p $PFASTQS
mkdir -p $FASTQCOUT
mkdir -p $STARDIR
mkdir -p $FCDIR
mkdir -p $TFASTQS
mkdir -p $TPFASTQS
mkdir -p $TFASTQCOUT
'''

out = run_cmd(cmd)

In [11]:
cmd = '''
$SRABINDIR/prefetch --option-file $WORKDIR/SraAccList.txt
'''
out = run_cmd(cmd)


2018-01-26T13:32:32 prefetch.2.8.2: 1) 'ERR899429' is found locally

2018-01-26T13:32:32 prefetch.2.8.2: 2) 'ERR899433' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 3) 'ERR899427' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 4) 'ERR899421' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 5) 'ERR899425' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 6) 'ERR899435' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 7) 'ERR899430' is found locally

2018-01-26T13:32:33 prefetch.2.8.2: 8) 'ERR899432' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 9) 'ERR899431' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 10) 'ERR899434' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 11) 'ERR899438' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 12) 'ERR899422' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 13) 'ERR899423' is found locally

2018-01-26T13:32:34 prefetch.2.8.2: 14) 'ERR899424' is found locally

2018-01-26T13:32:35 prefetch

In [12]:
cmd = '''
## Make STAR index if not exists
if [ ! -d "$STARGENOMEDIR/" ]; then
    echo "STAR index does not exist, building STAR index"

    mkdir $STARGENOMEDIR

    $BINDIR/STAR \
        --runThreadN $MAXCORES \
        --runMode genomeGenerate \
        --genomeDir $STARGENOMEDIR \
        --genomeFastaFiles $WHOLEGENOMEDIR/genome.fa \
        --sjdbGTFfile $GENEANNOTATIONDIR/genes.gtf \
        --sjdbOverhang 100
fi
'''
out = run_cmd(cmd)

In [24]:
cmd = '''
is_paired() {
    # function to examine whether a .sra file is paired-end sequencing
    # ref: https://www.biostars.org/p/139422/
    local SRA="$1"
    local x=$(
        $SRABINDIR/fastq-dump -I -X 1 -Z --split-spot "$SRA" 2>/dev/null \
        | awk '{if(NR % 2 == 1) print substr($1,length($1),1)}' \
        | uniq \
        | wc -l
    )
    # echo $SRA $x
    # $x should be 2 if paired-end, 1 if single-end
    if [ $x == 2 ]; then
        return 0 # true
    else
        return 1 # false
    fi
}

dump_sra() {
    # function to dump sra to fastq file with auto-detecting
    # whether reads are from single-end or paired-end
    local sra="$1"
    echo $sra
    echo $SRABINDIR
    if is_paired $sra; then
        echo "$sra is detected as paired-end sequencing reads"
        # Note that paired-end sequencing reads should be dumped into two fastq files
        $SRABINDIR/fastq-dump --gzip -I --split-files -O paired_fastqs "$sra"
    else
        echo "$sra is detected as single-end sequencing reads"
        $SRABINDIR/fastq-dump --gzip -O fastqs "$sra"
    fi
}


## Dump .sra to .fastq
echo "Dumping .sra files to .fastq.gz files"

## Dump in parallel using xargs
export -f is_paired
export -f dump_sra

find $DATA/*.sra | xargs --max-args=1 --max-procs=$MAXCORES -I {} bash -c 'dump_sra "$@"' _ {}
'''
out = run_cmd(cmd)


Dumping .sra files to .fastq.gz files
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899421.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899422.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899423.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899424.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899425.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899426.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899427.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899428.sra
/fs/project/PAS1307/bin/sratoolkit.2.8.2-1-ubuntu64/bin
/users/PAS1307/osu8805/rnaseq_2/data/sra/ERR899429.sra
/fs/project/PAS1307

In [50]:
old_cmd='''
#Activate the environment where cutadapt is installed.
source activate tophat
module load fastqc

#Quality filtering and adapter trimming
PHRED="20"

for basename in $(ls fastqs | cut -f1 -d '.'); do
    fq1=".fastq.gz"
    fq1=$FASTQS/$basename$fq1
    $TGBINDIR/trim_galore \
        -q $PHRED \
        -o $TFASTQS \
        $fq1
done

for basename in $(ls paired_fastqs | cut -f1 -d '_' | sort | uniq); do
    fq1="_1.fastq.gz"
    fq2="_2.fastq.gz"
    fq1=$PFASTQS/$basename$fq1
    fq2=$PFASTQS/$basename$fq2
    fqs="$fq1 $fq2"
    $TGBINDIR/trim_galore \
        -q $PHRED \
        --paired \
        -o $TPFASTQS/ \
        $fqs
done
'''

cmds = []
fastq_files = glob.glob(os.environ['FASTQS']+"/*.fastq.gz") 

for fastq_file in fastq_files:
    cmds.append("source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs "+fastq_file)
print(cmds)
    

pool = Pool(processes=MAXCORES)
results = pool.map_async(run_cmd, cmds)
pool.close()
pool.join()
pool.terminate()



fastqs
['source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899426.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899430.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899435.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899431.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899428.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899436.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899425.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899427.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR899433.fastq.gz', 'source activate rnaseq && $TGBINDIR/trim_galore -q 20 -o trimmed_fastqs fastqs/ERR

In [75]:
#fastqc of sequencing reads

old_cmd = '''
#load fastqc
module load fastqc

fqs=""

for basename in $(ls $FASTQS | cut -f1 -d '.'); do
    #basename=$(echo $fq | cut -f1 -d '.')
    fqd=$FASTQS
    fq1=".fastq.gz"
    fq1=$fqd/$basename$fq1
    fqs="$fqs $fq1"
    
done

for basename in $(ls $PFASTQS | cut -f1 -d '_' | sort | uniq); do
    echo $basename
    fqd=$PFASTQS
    fq1="_1.fastq.gz"
    fq2="_2.fastq.gz"
    fq1=$fqd/$basename$fq1
    fq2=$fqd/$basename$fq2
    fqs="$fqs $fq1 $fq2"
done

echo "Performing FastQC for $fqs"
fastqc --threads $MAXCORES $fqs -o $FASTQCOUT
'''
#out = run_cmd(cmd)

fastq_files = glob.glob(FASTQS+"/*.fastq.gz") 
print(fastq_files)
cmds = []
for fastq_file in fastq_files:
    cmds.append(ENVCMD + " && " +
                "module load fastqc && " +
                "fastqc " + 
                fastq_file +
                " -o " + FASTQCOUT)

pool = Pool(processes=MAXCORES)
results = pool.map_async(run_cmd, cmds)
pool.close()
pool.join()
pool.terminate()

['fastqs/ERR899426.fastq.gz', 'fastqs/ERR899430.fastq.gz', 'fastqs/ERR899435.fastq.gz', 'fastqs/ERR899431.fastq.gz', 'fastqs/ERR899428.fastq.gz', 'fastqs/ERR899436.fastq.gz', 'fastqs/ERR899425.fastq.gz', 'fastqs/ERR899427.fastq.gz', 'fastqs/ERR899433.fastq.gz', 'fastqs/ERR899438.fastq.gz', 'fastqs/ERR899437.fastq.gz', 'fastqs/ERR899422.fastq.gz', 'fastqs/ERR899432.fastq.gz', 'fastqs/ERR899434.fastq.gz', 'fastqs/ERR899421.fastq.gz', 'fastqs/ERR899423.fastq.gz', 'fastqs/ERR899424.fastq.gz', 'fastqs/ERR899429.fastq.gz']
Started analysis of ERR899438.fastq.gz
Started analysis of ERR899425.fastq.gz
Started analysis of ERR899435.fastq.gz
Started analysis of ERR899426.fastq.gz
Started analysis of ERR899429.fastq.gz
Started analysis of ERR899423.fastq.gz
Started analysis of ERR899433.fastq.gz
Started analysis of ERR899421.fastq.gz
Started analysis of ERR899424.fastq.gz
Started analysis of ERR899428.fastq.gz
Started analysis of ERR899431.fastq.gz
Started analysis of ERR899422.fastq.gz
Started a

In [70]:
old_cmd = '''
#fastqc of trimmed_sequencing reads

#load fastqc
module load fastqc

fqs=""

for basename in $(ls $FASTQS | cut -f1 -d '.'); do
    #basename=$(echo $fq | cut -f1 -d '.')
    fqd=$TFASTQS
    fq1=".fastq.gz"
    fq1=$fqd/$basename$fq1
    fqs="$fqs $fq1"
    
done

for basename in $(ls $P$FASTQS | cut -f1 -d '_' | sort | uniq); do
    echo $basename
    fqd=$PTFASTQS
    fq1="_trimmed_1.fastq.gz"
    fq2="_trimmed_2.fastq.gz"
    fq1=$fqd/$basename$fq1
    fq2=$fqd/$basename$fq2
    fqs="$fqs $fq1 $fq2"
done

echo "Performing FastQC for $fqs"
fastqc --threads $MAXCORES $fqs -o $TFASTQCOUT
'''

#out=run_cmd(cmd)

tfastq_files = glob.glob(TFASTQS+"/*.fq.gz") 

cmds = []
for tfastq_file in tfastq_files:
    cmds.append(ENVCMD + " && " +
                "module load fastqc && " +
                "fastqc " + 
                tfastq_file +
                " -o " + TFASTQCOUT)

pool = Pool(processes=MAXCORES)
results = pool.map_async(run_cmd, cmds)
pool.close()
pool.join()
pool.terminate()

Started analysis of ERR899435_trimmed.fq.gz
Started analysis of ERR899425_trimmed.fq.gz
Started analysis of ERR899431_trimmed.fq.gz
Started analysis of ERR899434_trimmed.fq.gz
Started analysis of ERR899432_trimmed.fq.gz
Started analysis of ERR899423_trimmed.fq.gz
Started analysis of ERR899429_trimmed.fq.gz
Started analysis of ERR899424_trimmed.fq.gz
Started analysis of ERR899428_trimmed.fq.gz
Started analysis of ERR899436_trimmed.fq.gz
Started analysis of ERR899422_trimmed.fq.gz
Started analysis of ERR899427_trimmed.fq.gz
Started analysis of ERR899421_trimmed.fq.gz
Started analysis of ERR899437_trimmed.fq.gz
Started analysis of ERR899433_trimmed.fq.gz
Started analysis of ERR899430_trimmed.fq.gz
Started analysis of ERR899438_trimmed.fq.gz
Started analysis of ERR899426_trimmed.fq.gz
Approx 5% complete for ERR899427_trimmed.fq.gz
Approx 5% complete for ERR899428_trimmed.fq.gz
Approx 5% complete for ERR899435_trimmed.fq.gz
Approx 5% complete for ERR899421_trimmed.fq.gz
Approx 5% complete f

In [48]:
cmd='''
#module load star

for basename in $(ls $FASTQC | cut -f1 -d '.'); do
    fqd=$TFASTQS
    fq1=".fastq.gz"
    fq1=$fqd/$basename$fq1
    
    echo "Aligning reads from $basename to the reference genome"
    $BINDIR/STAR \
        --genomeDir $STARGENOMEDIR \
        --sjdbGTFfile $GENEANNOTATIONDIR/genes.gtf \
        --runThreadN $MAXCORES \
        --outSAMstrandField intronMotif \
        --outFilterIntronMotifs RemoveNoncanonical \
        --outFileNamePrefix $STARDIR/$basename  \
        --readFilesIn $fq1 \
        --readFilesCommand zcat \
        --outSAMtype BAM Unsorted \
        --outReadsUnmapped Fastx \
        --outSAMmode Full
done


for basename in $(ls $PFASTQC | cut -f1 -d '_' | sort | uniq); do
    fqd=$TPFASTQS
    fq1="trimmed_1.fastq.gz"
    fq2="trimmed_2.fastq.gz"
    fq1=$fqd/$basename$fq1
    fq2=$fqd/$basename$fq2
    fqs="$fqs $fq1 $fq2"
    
    
    echo "Aligning reads from $basename to the reference genome"
    $BINDIR/STAR \
        --genomeDir $STARGENOMEDIR \
        --sjdbGTFfile $GENEANNOTATIONDIR/genes.gtf \
        --runThreadN $MAXCORES \
        --outSAMstrandField intronMotif \
        --outFilterIntronMotifs RemoveNoncanonical \
        --outFileNamePrefix $STARDIR/$basename \
        --readFilesIn $fq1 $fq2 \
        --readFilesCommand zcat \
        --outSAMtype BAM Unsorted \
        --outReadsUnmapped Fastx \
        --outSAMmode Full
done
'''
#out=run_cmd(cmd)





Aligning reads from ERR899421 to the reference genome
Jan 19 12:19:38 ..... started STAR run
Jan 19 12:19:38 ..... loading genome
Jan 19 12:19:52 ..... processing annotations GTF
Jan 19 12:20:25 ..... inserting junctions into the genome indices
Jan 19 12:21:33 ..... started mapping
Jan 19 12:23:17 ..... finished successfully
Aligning reads from ERR899422 to the reference genome
Jan 19 12:23:43 ..... started STAR run
Jan 19 12:23:44 ..... loading genome
Jan 19 12:23:57 ..... processing annotations GTF
Jan 19 12:24:06 ..... inserting junctions into the genome indices
Jan 19 12:25:13 ..... started mapping
Jan 19 12:26:56 ..... finished successfully
Aligning reads from ERR899423 to the reference genome
Jan 19 12:27:25 ..... started STAR run
Jan 19 12:27:25 ..... loading genome
Jan 19 12:27:38 ..... processing annotations GTF
Jan 19 12:28:12 ..... inserting junctions into the genome indices
Jan 19 12:29:19 ..... started mapping
Jan 19 12:30:59 ..... finished successfully
Aligning reads from

In [54]:
cmd='''
for basename in $(find star_output/ -name '*Aligned.out.bam' -exec basename {} \; | cut -f1 -d '.' ); do
    suffix=".out.bam"
    outname="$FCDIR/$basename.count.txt"
    bam="$STARDIR/$basename$suffix"    
    $BINDIR/featureCounts \
        -T $MAXCORES \
        -t exon \
        -g gene_id \
        -a  $GENEANNOTATIONDIR/genes.gtf \
        -o $outname \
        $bam 
done
'''
out=run_cmd(cmd)


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v1.6.0

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                           S ./star_output/ERR899421Aligned.out.bam         ||
||                                                                            ||
||             Output file : ./featureCount_output/ERR899421Aligned.count ... ||
||                 Summary : ./featureCount_output/ERR899421Align

In [ ]:
cmd='''
bams=""
for basename in $(find star_output/ -name '*Aligned.out.bam' -exec basename {} \; | cut -f1 -d '.' ); do
    suffix=".out.bam"
    outname="combined.counts.txt"
    bam="$STARDIR/$basename$suffix"    
    bams="$bams $bam"
done

$BINDIR/featureCounts \
        -T $MAXCORES \
        -t exon \
        -g gene_id \
        -a  $GENEANNOTATIONDIR/genes.gtf \
        -o $outname \
        $bams
'''
out=run_cmd(cmd)

In [ ]:
cmd='''
#multiqc of sequencing reads

source activate tophat
multiqc .
'''
out=run_cmd(cmd)

In [76]:
from IPython.display import FileLinks, display

d = FileLinks(os.environ['WORKDIR'], included_suffixes=['.html'])
e = FileLinks(os.path.join(os.environ['WORKDIR'], os.environ['FASTQCOUT']), included_suffixes=['.html'])
f = FileLinks(os.path.join(os.environ['WORKDIR'], os.environ['TFASTQCOUT']), included_suffixes=['.html'])

display(d)
display(e)
display(f)

/users/PAS1307/osu8805/rnaseq_2/fastQC_output/
  ERR899423_fastqc.html
  ERR899424_fastqc.html
  ERR899428_fastqc.html
  ERR899427_fastqc.html
  ERR899421_fastqc.html
  ERR899435_fastqc.html
  ERR899432_fastqc.html
  ERR899422_fastqc.html
  ERR899436_fastqc.html
  ERR899434_fastqc.html
  ERR899430_fastqc.html
  ERR899437_fastqc.html
  ERR899426_fastqc.html
  ERR899429_fastqc.html
  ERR899431_fastqc.html
  ERR899433_fastqc.html
  ERR899425_fastqc.html
  ERR899438_fastqc.html
/users/PAS1307/osu8805/rnaseq_2/trimmed_fastQC_output/
  ERR899427_fastqc.html
  ERR899428_fastqc.html
  ERR899430_fastqc.html
  ERR899421_fastqc.html
  ERR899423_fastqc.html
  ERR899432_fastqc.html
  ERR899422_fastqc.html
  ERR899435_fastqc.html
  ERR899424_fastqc.html
  ERR899436_fastqc.html
  ERR899434_fastqc.html
  ERR899437_fastqc.html
  ERR899429_fastqc.html
  ERR899431_fastqc.html
  ERR899426_fastqc.html
  ERR899433_fastqc.html
  ERR899425_fastqc.html
  ERR899438_fastqc.html
  ERR899428_trimmed_fastqc.html
  ERR899427_trimmed_fastqc.html
  ERR899435_trimmed_fastqc.html
  ERR899421_trimmed_fastqc.html
  ERR899432_trimmed_fastqc.html
  ERR899436_trimmed_fastqc.html
  ERR899434_trimmed_fastqc.html
  ERR899423_trimmed_fastqc.html
  ERR899424_trimmed_fastqc.html
  ERR899437_trimmed_fastqc.html
  ERR899422_trimmed_fastqc.html
  ERR899430_trimmed_fastqc.html
  ERR899426_trimmed_fastqc.html
  ERR899429_trimmed_fastqc.html
  ERR899431_trimmed_fastqc.html
  ERR899433_trimmed_fastqc.html
  ERR899425_trimmed_fastqc.html
  ERR899438_trimmed_fastqc.html

/users/PAS1307/osu8805/rnaseq_2/fastQC_output/
  ERR899423_fastqc.html
  ERR899424_fastqc.html
  ERR899428_fastqc.html
  ERR899427_fastqc.html
  ERR899421_fastqc.html
  ERR899435_fastqc.html
  ERR899432_fastqc.html
  ERR899422_fastqc.html
  ERR899436_fastqc.html
  ERR899434_fastqc.html
  ERR899430_fastqc.html
  ERR899437_fastqc.html
  ERR899426_fastqc.html
  ERR899429_fastqc.html
  ERR899431_fastqc.html
  ERR899433_fastqc.html
  ERR899425_fastqc.html
  ERR899438_fastqc.html

/users/PAS1307/osu8805/rnaseq_2/trimmed_fastQC_output/
  ERR899427_fastqc.html
  ERR899428_fastqc.html
  ERR899430_fastqc.html
  ERR899421_fastqc.html
  ERR899423_fastqc.html
  ERR899432_fastqc.html
  ERR899422_fastqc.html
  ERR899435_fastqc.html
  ERR899424_fastqc.html
  ERR899436_fastqc.html
  ERR899434_fastqc.html
  ERR899437_fastqc.html
  ERR899429_fastqc.html
  ERR899431_fastqc.html
  ERR899426_fastqc.html
  ERR899433_fastqc.html
  ERR899425_fastqc.html
  ERR899438_fastqc.html
  ERR899428_trimmed_fastqc.html
  ERR899427_trimmed_fastqc.html
  ERR899435_trimmed_fastqc.html
  ERR899421_trimmed_fastqc.html
  ERR899432_trimmed_fastqc.html
  ERR899436_trimmed_fastqc.html
  ERR899434_trimmed_fastqc.html
  ERR899423_trimmed_fastqc.html
  ERR899424_trimmed_fastqc.html
  ERR899437_trimmed_fastqc.html
  ERR899422_trimmed_fastqc.html
  ERR899430_trimmed_fastqc.html
  ERR899426_trimmed_fastqc.html
  ERR899429_trimmed_fastqc.html
  ERR899431_trimmed_fastqc.html
  ERR899433_trimmed_fastqc.html
  ERR899425_trimmed_fastqc.html
  ERR899438_trimmed_fastqc.html